In [1]:
# Parameters
adataPATH_ref = "03_downstream_analysis/08_PatientClassifier/scANVI/results/reference/scANVI_VALIDATION_*_100_Level2_*.h5ad"
adataPATH_query = "03_downstream_analysis/08_PatientClassifier/scANVI/results/query/scANVI_VALIDATION_*_100_Level2_*.h5ad"


In [2]:
for v in ['adataPATH_ref','adataPATH_query']:
    if v in locals() or v in globals():
        print(f"{v} = {eval(v)}")
    else:
        raise Exception(f"{v} not specified")

adataPATH_ref = 03_downstream_analysis/08_PatientClassifier/scANVI/results/reference/scANVI_VALIDATION_*_100_Level2_*.h5ad
adataPATH_query = 03_downstream_analysis/08_PatientClassifier/scANVI/results/query/scANVI_VALIDATION_*_100_Level2_*.h5ad


In [3]:
import os
import sys
from glob import glob

import scanpy as sc
import pandas as pd

import pynndescent
import numpy as np
import numba

from pyprojroot import here

sys.path.insert(1, str(here('bin')))
# Import custom functions
from customPythonFunctions import aggregating_features

### Defining kNN label transfer function

In [4]:
class LabelTransferWithKNN:
    """ See https://docs.scvi-tools.org/en/stable/tutorials/notebooks/scrna/query_hlca_knn.html """
    def fit(self, ref_adata):
        self.ref_adata = ref_adata
        self.nn_index = pynndescent.NNDescent(self.ref_adata.X)
        self.nn_index.prepare()
        return self

    def predict(self, query_adata, label_keys: list[str] = ['Level1']):

        ref_neighbors, ref_distances = self.nn_index.query(query_adata.X)

        # convert distances to affinities
        stds = np.std(ref_distances, axis=1)
        stds = (2.0 / stds) ** 2
        stds = stds.reshape(-1, 1)
        ref_distances_tilda = np.exp(-np.true_divide(ref_distances, stds))
        weights = ref_distances_tilda / np.sum(ref_distances_tilda, axis=1, keepdims=True)

        for l in label_keys:
            ref_cats = self.ref_adata.obs[l].cat.codes.to_numpy()[ref_neighbors]
            p, u = self.weighted_prediction(weights, ref_cats)
            p = np.asarray(self.ref_adata.obs[l].cat.categories)[p]
            query_adata.obs[l + "_pred"], query_adata.obs[l + "_uncertainty"] = p, u
        
        return query_adata
    
    @staticmethod
    @numba.njit
    def weighted_prediction(weights, ref_cats):
        """Get highest weight category."""
        N = len(weights)
        predictions = np.zeros((N,), dtype=ref_cats.dtype)
        uncertainty = np.zeros((N,))
        for i in range(N):
            obs_weights = weights[i]
            obs_cats = ref_cats[i]
            best_prob = 0
            for c in np.unique(obs_cats):
                cand_prob = np.sum(obs_weights[obs_cats == c])
                if cand_prob > best_prob:
                    best_prob = cand_prob
                    predictions[i] = c
                    uncertainty[i] = max(1 - best_prob, 0)

        return predictions, uncertainty

### Loading data

In [5]:
adataPATH_ref
adataPATH_ref_list = glob(str(here(adataPATH_ref)))
assert(len(adataPATH_ref_list) == 1)

adataPATH_query
adataPATH_query_list = glob(str(here(adataPATH_query)))
assert(len(adataPATH_query_list) == 1)

In [6]:
adataR = sc.read_h5ad(adataPATH_ref_list[0])
adataQ = sc.read_h5ad(adataPATH_query_list[0])
adataR, adataQ

(AnnData object with n_obs × n_vars = 4435922 × 100
     obs: 'studyID', 'libraryID', 'sampleID', 'chemistry', 'disease', 'sex', 'binned_age', 'Level1', 'Level2', '_scvi_batch', '_scvi_labels',
 AnnData object with n_obs × n_vars = 849922 × 100
     obs: 'studyID', 'libraryID', 'sampleID', 'chemistry', 'technology', 'disease', 'sex', 'binned_age', '_scvi_batch', 'Level2', '_scvi_labels', 'labels')

In [7]:
Level1_dict = adataR.obs[['Level1','Level2']].set_index('Level2').to_dict()['Level1']

In [8]:
adataQ.obs['Level1_scANVIpredict'] = adataQ.obs['labels'].map(Level1_dict)

In [9]:
adataQ.obs['Level1_scANVIpredict'].value_counts(normalize = True)

Level1_scANVIpredict
Mono              0.264116
T_CD4_NonNaive    0.188356
T_CD4_Naive       0.138802
T_CD8_NonNaive    0.123374
ILC               0.086256
B                 0.081290
T_CD8_Naive       0.054317
UTC               0.027524
DC                0.013737
Plasma            0.006108
Cycling_cells     0.005575
pDC               0.005320
Platelets         0.004325
Progenitors       0.000802
RBC               0.000099
Name: proportion, dtype: float64

#### Label transfer with kNN

In [10]:
kNNclf = LabelTransferWithKNN().fit(adataR)

In [11]:
adataQ = kNNclf.predict(query_adata = adataQ, label_keys = ['Level1'])
adataQ

AnnData object with n_obs × n_vars = 849922 × 100
    obs: 'studyID', 'libraryID', 'sampleID', 'chemistry', 'technology', 'disease', 'sex', 'binned_age', '_scvi_batch', 'Level2', '_scvi_labels', 'labels', 'Level1_scANVIpredict', 'Level1_pred', 'Level1_uncertainty'

In [12]:
adataQ.obs['Level1_kNN_pred'] = adataQ.obs['Level1_pred']

In [13]:
adataQ.obs['Level1_kNN_pred'].value_counts(normalize = True)

Level1_kNN_pred
Mono              0.258004
T_CD4_Naive       0.163183
T_CD4_NonNaive    0.160361
T_CD8_NonNaive    0.118709
ILC               0.084814
B                 0.081516
T_CD8_Naive       0.047023
Platelets         0.025632
UTC               0.025371
DC                0.012600
Cycling_cells     0.010560
Plasma            0.005280
pDC               0.005095
Progenitors       0.001685
RBC               0.000169
Name: proportion, dtype: float64

In [14]:
from sklearn.metrics import adjusted_rand_score as ari

In [15]:
ari(adataQ.obs['Level1_kNN_pred'],adataQ.obs['Level1_scANVIpredict'])

0.8601664045803622

#### Generating PSEUDOBULKs

In [16]:
adataPB_R = aggregating_features(Z = adataR.X, 
                             obsDF = adataR.obs[['sampleID','Level1','disease']], 
                             mode = 'mean', 
                             obs_names_col=['sampleID','Level1'], 
                             min_observation=0)
adataPB_R

AnnData object with n_obs × n_vars = 11372 × 100
    obs: 'sampleID', 'Level1', 'disease', 'n_observation'

**Considering scANVI predicted labels**

In [17]:
adataPB_Q_scANVI = aggregating_features(Z = adataQ.X, 
                             obsDF = adataQ.obs[['sampleID','Level1_scANVIpredict','disease']], 
                             mode = 'mean', 
                             obs_names_col=['sampleID','Level1_scANVIpredict'], 
                             min_observation=0)

adataPB_Q_scANVI.obs.rename({'Level1_scANVIpredict':'Level1'}, axis=1, inplace=True)
adataPB_Q_scANVI

AnnData object with n_obs × n_vars = 1943 × 100
    obs: 'sampleID', 'Level1', 'disease', 'n_observation'

In [18]:
adataPB_Q_scANVI.obs.Level1.value_counts()

Level1
B                 144
ILC               144
Mono              144
T_CD4_Naive       144
T_CD4_NonNaive    144
T_CD8_NonNaive    144
UTC               144
DC                143
Cycling_cells     142
T_CD8_Naive       142
pDC               137
Plasma            130
Platelets         122
Progenitors       107
RBC                12
Name: count, dtype: int64

**Considering labels transferred with kNN**

In [19]:
adataPB_Q_kNN = aggregating_features(Z = adataQ.X, 
                             obsDF = adataQ.obs[['sampleID','Level1_kNN_pred','disease']], 
                             mode = 'mean', 
                             obs_names_col=['sampleID','Level1_kNN_pred'], 
                             min_observation=0)
adataPB_Q_kNN.obs.rename({'Level1_kNN_pred':'Level1'}, axis=1, inplace=True)
adataPB_Q_kNN

AnnData object with n_obs × n_vars = 1988 × 100
    obs: 'sampleID', 'Level1', 'disease', 'n_observation'

In [20]:
adataPB_Q_kNN.obs.Level1.value_counts()

Level1
B                 144
Cycling_cells     144
ILC               144
Mono              144
T_CD4_Naive       144
T_CD4_NonNaive    144
T_CD8_NonNaive    144
UTC               144
DC                143
T_CD8_Naive       142
Platelets         141
pDC               137
Plasma            129
Progenitors       125
RBC                19
Name: count, dtype: int64

### Saving pseudobulk adata objects

In [21]:
adataPATH_ref_list[0]

'/scratch_isilon/groups/singlecell/shared/projects/Inflammation-PBMCs-Atlas/03_downstream_analysis/08_PatientClassifier/scANVI/results/reference/scANVI_VALIDATION_256_100_Level2_run1_finetuning.h5ad'

In [22]:
adataPATH_ref_list[0].replace('/reference/scANVI_','/PSEUDOBULKs/scANVI_PSEUDOBULK_reference_')

'/scratch_isilon/groups/singlecell/shared/projects/Inflammation-PBMCs-Atlas/03_downstream_analysis/08_PatientClassifier/scANVI/results/PSEUDOBULKs/scANVI_PSEUDOBULK_reference_VALIDATION_256_100_Level2_run1_finetuning.h5ad'

In [23]:
adataPB_R.write(here(adataPATH_ref_list[0].replace('/reference/scANVI_','/PSEUDOBULKs/scANVI_PSEUDOBULK_reference_')), compression='gzip')

In [24]:
adataPB_Q_scANVI.write(here(adataPATH_query_list[0].replace('/query/scANVI_','/PSEUDOBULKs/scANVI_PSEUDOBULK_query_')), compression='gzip')

In [25]:
adataPB_Q_kNN.write(here(adataPATH_query_list[0].replace('/query/scANVI_','/PSEUDOBULKs/scANVI_PSEUDOBULK_kNN_query_')), compression='gzip')